In [1]:
import os
import shutil

train_dir = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\images'
dest_dir = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\all_images'
counter = 0

for subdir, dirs, files in os.walk(train_dir):
    #print(files)
    for file in files:
        full_path = os.path.join(subdir, file)
        shutil.copy(full_path, dest_dir)
        counter = counter + 1
print(counter)

20580


In [2]:
import numpy as np

subdirs, dirs, files = os.walk(r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\all_images').__next__()
m = len(files)
print(m)

filenames = []
labels = np.zeros((m, 1))

images_dir = r'C:\Users\haris\AI\ML\Projects\Data\Dog Breed Classification\all_images'
filenames_counter = 0
labels_counter = -1

for subdir, dirs, files in os.walk(train_dir):
    #print(files)
    for file in files:
        filenames.append(file)
        labels[filenames_counter, 0] = labels_counter
        filenames_counter = filenames_counter + 1
    labels_counter = labels_counter+1
    
print(len(filenames))
print(labels.shape)

20580
20580
(20580, 1)


In [3]:
# saving the filename array as .npy file
np.save('filenames.npy', filenames)

import keras
from keras.utils import to_categorical

# One hot vector representation of labels
y_labels_one_hot = to_categorical(labels)

# saving the y_labels_one_hot array as a .npy file
np.save('y_labels_one_hot.npy', y_labels_one_hot)

In [4]:
from sklearn.utils import shuffle

filenames_shuffled, y_labels_one_hot_shuffled = shuffle(filenames, y_labels_one_hot)

# saving the shuffled file.
# you can load them later using np.load().
np.save('y_labels_one_hot_shuffled.npy', y_labels_one_hot_shuffled)
np.save('filenames_shuffled.npy', filenames_shuffled)

In [5]:
from sklearn.model_selection import train_test_split

# Used this line as our filename array is not a numpy array.
filenames_shuffled_numpy = np.array(filenames_shuffled)

X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(
    filenames_shuffled_numpy, y_labels_one_hot_shuffled, test_size=0.2, random_state=1)

print(X_train_filenames.shape) # (16464,)
print(y_train.shape)           # (16464, 120)

print(X_val_filenames.shape)   # (4116,)
print(y_val.shape)             # (4116, 120)

# You can save these files as well. As you will be using them later for training and validation of your model.
np.save('X_train_filenames.npy', X_train_filenames)
np.save('y_train.npy', y_train)

np.save('X_val_filenames.npy', X_val_filenames)
np.save('y_val.npy', y_val)

(16464,)
(16464, 120)
(4116,)
(4116, 120)


In [6]:
class My_Custom_Generator(keras.utils.Sequence) :
    
    def __init__(self, image_filenames, labels, batch_size) :
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
    
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)

    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

        return np.array([
                resize(imread(r'C:/Users/haris/AI/ML/Projects/Data/Dog Breed Classification/all_images/' + str(file_name)), (224, 224, 3))
                   for file_name in batch_x])/255.0, np.array(batch_y)

In [8]:
batch_size = 32

my_training_batch_generator = My_Custom_Generator(X_train_filenames, y_train, batch_size)
my_validation_batch_generator = My_Custom_Generator(X_val_filenames, y_val, batch_size)

In [9]:
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten

from keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
from functools import partial

import os
import xml.etree.ElementTree as ET

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.applications import Xception
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
AUTO = tf.data.experimental.AUTOTUNE

In [10]:
# Specify input shape and number of classes
input_shape = (224, 224, 3)
num_classes=120

# Build Simple CNN
def build_simple_cnn(input_shape, num_classes):
    model = Sequential()
    
    # Convolutional blocks
    for _ in range(5):
        model.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(ReLU())
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(0.25))
    
    # Flatten and fully connected layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [11]:
# Build Xception model with transfer learning
def build_xception(input_shape, num_classes):
    base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Build and compile Simple CNN model
simple_cnn_model = build_simple_cnn(input_shape, num_classes)
simple_cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Build and compile Xception model
xception_model = build_xception(input_shape, num_classes)
xception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Data augmentation for Simple CNN
# datagen = tf.keras.preprocessing.image.ImageDataGenerator(
#     horizontal_flip=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     fill_mode='nearest',
#     preprocessing_function=tf.keras.applications.xception.preprocess_input
# )

# # Create data generators using tf.data
# def create_data_generator(images, labels, batch_size, is_training=True):
#     dataset = tf.data.Dataset.from_tensor_slices((images, labels))
#     if is_training:
#         dataset = dataset.shuffle(buffer_size=len(images)).repeat()

#     dataset = dataset.map(lambda x, y: (datagen.random_transform(x), y), num_parallel_calls=tf.data.AUTOTUNE)
#     dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
#     return dataset

# # Create data generators
# train_data_generator = create_data_generator(X_train, y_train_onehot, batch_size=32)
# val_data_generator = create_data_generator(X_val, y_val_onehot, batch_size=32, is_training=False)
# test_data_generator = create_data_generator(X_test, y_test_onehot, batch_size=32, is_training=False)

# Train Simple CNN
history_simple_cnn = simple_cnn_model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(16464 // batch_size),
                   epochs = 10,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(4116 // batch_size))

# Train Xception
history_xception = xception_model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(16464 // batch_size),
                   epochs = 10,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(4116 // batch_size))

Epoch 1/10
514/514 [==============================] - 467s 891ms/step - loss: 4.7999 - accuracy: 0.0093 - val_loss: 4.7848 - val_accuracy: 0.0132
Epoch 2/10
514/514 [==============================] - 396s 770ms/step - loss: 4.7839 - accuracy: 0.0120 - val_loss: 4.7834 - val_accuracy: 0.0132
Epoch 3/10
514/514 [==============================] - 396s 771ms/step - loss: 4.7819 - accuracy: 0.0118 - val_loss: 4.7826 - val_accuracy: 0.0132
Epoch 4/10
514/514 [==============================] - 428s 832ms/step - loss: 4.7809 - accuracy: 0.0117 - val_loss: 4.7823 - val_accuracy: 0.0132
Epoch 5/10
514/514 [==============================] - 391s 762ms/step - loss: 4.7803 - accuracy: 0.0120 - val_loss: 4.7823 - val_accuracy: 0.0132
Epoch 6/10
514/514 [==============================] - 420s 818ms/step - loss: 4.7799 - accuracy: 0.0113 - val_loss: 4.7822 - val_accuracy: 0.0132
Epoch 7/10
514/514 [==============================] - 431s 839ms/step - loss: 4.7798 - accuracy: 0.0120 - val_loss: 4.7823 -

KeyboardInterrupt: 

In [ ]:
# # Evaluate models
# simple_cnn_loss, simple_cnn_acc = simple_cnn_model.evaluate(X_test, y_test, verbose=2)
# xception_loss, xception_acc = xception_model.evaluate(X_test, y_test, verbose=2)

# print("Simple CNN - Test accuracy:", simple_cnn_acc)
# print("Xception - Test accuracy:", xception_acc)